In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [8]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [11]:
X = df.drop(columns=['math score'], axis=1)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [12]:
y = df['math score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [13]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features),
    ]
)

In [14]:
X = preprocessor.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [19]:
def evaluate_model(true, predicted):
    mae =  mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mse, rmse, r2_square

In [24]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'K-Neighbors Regressors': KNeighborsRegressor(),
    'Decision Trees': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.values())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print('- Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
    print('- R2 Score: {:.4f}'.format(model_train_r2))
    print('- Mean Absolute Error: {:.4f}'.format(model_train_mae))

    print('----------------------------------------')

    print('Model performance for Testing set')
    print('- Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
    print('- Mean Absolute Error: {:.4f}'.format(model_test_mae))
    print('- R2 Score: {:.4f}'.format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

    

LinearRegression()
Model performance for Training set
- Root Mean Squared Error: 5.3244
- R2 Score: 0.8743
- Mean Absolute Error: 28.3487
----------------------------------------
Model performance for Testing set
- Root Mean Squared Error: 5.3960
- Mean Absolute Error: 29.1167
- R2 Score: 0.8803


Lasso()
Model performance for Training set
- Root Mean Squared Error: 6.5938
- R2 Score: 0.8071
- Mean Absolute Error: 43.4784
----------------------------------------
Model performance for Testing set
- Root Mean Squared Error: 6.5197
- Mean Absolute Error: 42.5064
- R2 Score: 0.8253


Ridge()
Model performance for Training set
- Root Mean Squared Error: 5.3233
- R2 Score: 0.8743
- Mean Absolute Error: 28.3378
----------------------------------------
Model performance for Testing set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 29.0563
- R2 Score: 0.8806


KNeighborsRegressor()
Model performance for Training set
- Root Mean Squared Error: 5.7077
- R2 Score: 0.8555
- Mean Absolute

In [25]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=['R2_Score'],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880345
5,Random Forest Regressor,0.852450
7,CatBoostRegressor,0.851632
8,AdaBoostRegressor,0.847291
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressors,0.783813
4,Decision Trees,0.738348
